In [18]:
import numpy as np
import os
import json
import tensorflow as tf
import keras.backend as K
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout, Lambda
from VGGish.vggish import VGGish
import VGGish.vggish_params as params
from VGGish.vggish_input import wavfile_to_examples
from DataGenerator import ClassificationDataGenerator, FingerprintingDataGenerator
from ModelLogger import ModelLogger
np.random.seed(42)

In [4]:
models_folder = 'TrainedModels'
model_name = 'fingerprinting_VGGish_16_v0.1_flatten_6.h5'
model = load_model(os.path.join('.',models_folder,model_name))
model.summary()

Model: "fingerprinting_VGGish_16_v0.1_flatten_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Left_Input (InputLayer)         (None, 400, 64, 1)   0                                            
__________________________________________________________________________________________________
Right_Input (InputLayer)        (None, 400, 64, 1)   0                                            
__________________________________________________________________________________________________
Encoder (Model)                 (None, 16)           5318928     Left_Input[0][0]                 
                                                                 Right_Input[0][0]                
__________________________________________________________________________________________________
L1_Layer (Lambda)               (None, 16)           0      

In [31]:
in_folder = os.path.join('.','Spectrograms','test')
out_folder = os.path.join('.','GeneratedFingerprints')
os.makedirs(out_folder,exist_ok=True)
out_file_name = '.'.join(model_name.split('.')[:-1]) + '.json'
out_file = os.path.join(out_folder,out_file_name)
artists = os.listdir(in_folder)
batch_size = 128
data_gen = ClassificationDataGenerator(
    data_dir=in_folder,
    batch_size=batch_size,
    mode='multi-class',
    labels = artists
)

def get_label(labels, one_hot_vector):
    return labels[one_hot_vector.tolist().index(1)]

In [16]:
encoder = model.get_layer('Encoder')
all_fingerprints = {artist:[] for artist in artists}
for i in range(len(data_gen)):
    print(i+1,len(data_gen),end='\r')
    X,y = data_gen.__getitem__(i)
    fingerprints = encoder.predict(X)
    for i in range(batch_size):
        all_fingerprints[get_label(artists,y[i])].append(fingerprints[i])

IndexError: index 7 is out of bounds for axis 0 with size 7

In [32]:
def default(item):
    if isinstance(item,np.ndarray):
        return item.tolist()

avg_fingerprints = {k:np.average(all_fingerprints[k],axis=0) for k in all_fingerprints}
with open(out_file,'w') as f:
    json.dump({'all_fingerprints':all_fingerprints,'avg_fingerprints':avg_fingerprints},f,default=default)